# Piperack Generator

## Module imports and connection to STAAD

In [6]:
from base.helper.general import *
from base.staad_base.geometry import *
from base.structural_elements.beam import *
from base.structural_elements.column import *
from base.piperack.portal import *
from base.staad_base.load import *
from base.staad_base.design import *
from base.staad_base.property import *
from base.staad_base.optimise_member import *
from IPython.display import display, Markdown
from pyperclip import copy

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output

add_beams = add_beams(geometry=geometry)

## Inputs

### Steel

In [7]:
base_point_of_first_portal = Point3D(0,0,0)
width_of_piperack = 8
portal_distances = [0,8,16,24,30,36,42]
column_distances = [0,width_of_piperack]
tier_elevations = [3,6,9,12,14]
long_beam_elevations = [4.5,7.5,10.5,tier_elevations[-1]]
braces_placement = [False,True,False,False,True,False]

### Concrete

In [8]:
pedestal_height = 1


## Generator Code

### Primary Members

In [9]:
portals = []
longitudinal_beams = []
stubs = []
intermediate_transverse_beams = []
intermediate_long_beams = []
plan_braces = []
vertical_braces = []

portal_beam_ids = [] 
portal_column_ids = [] 
portal_pedestal_ids = [] 
long_beam_ids = [] 
stub_ids = []
intermediate_transverse_ids = []
intermediate_long_ids = []
plan_brace_ids = []
vertical_brace_ids = []

portal_count = len(portal_distances)
portal : PiperackPortal = PiperackPortal(base=base_point_of_first_portal) 


for column in column_distances:
    portal.add_column(Column3D(base=Point3D(column,0,0),height=tier_elevations[-1])) 
    portal.add_pedestal(Column3D(base=Point3D(column,0,0),height=-1*pedestal_height)) 
    
for tier in tier_elevations:
    for column_i in range(len(column_distances)-1):
        portal.add_beam(Beam3D(start=Point3D(column_distances[column_i],tier,0),end=Point3D(column_distances[column_i+1],tier,0))) 

portals = [portal.shift(Point3D(0,0,dist)) for dist in portal_distances]

for portal_i in range(len(portals)-1):
    for column in column_distances:
        for long_beam in long_beam_elevations:
            long_beam_x = Beam3D(start=Point3D(column,long_beam,portals[portal_i].base.z),end=Point3D(column,long_beam,portals[portal_i+1].base.z))
            longitudinal_beams.append(long_beam_x)


for portal in portals:
    portal_beam_ids = [*portal_beam_ids,*add_beams(portal.beams)]
    portal_column_ids = [*portal_column_ids,*add_beams(portal.columns)]
    portal_pedestal_ids = [*portal_pedestal_ids,*add_beams(portal.pedestals)]

long_beam_ids = add_beams(longitudinal_beams)

markdown_output = ''
markdown_output = "## Primary Members Created\n\n"
markdown_output += f"| Member Types     | IDs                  |\n"
markdown_output += f"| ---              | ---                  |\n"
markdown_output += f"| Portal Columns   | {portal_column_ids}   |\n"
markdown_output += f"| Portal Pedestals | {portal_pedestal_ids} |\n"
markdown_output += f"| Portal Beams     | {portal_beam_ids}     |\n"
markdown_output += f"| Long Beams       | {long_beam_ids}       |\n"

display(Markdown(markdown_output))

## Primary Members Created

| Member Types     | IDs                  |
| ---              | ---                  |
| Portal Columns   | [202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215]   |
| Portal Pedestals | [8, 9, 17, 18, 26, 27, 35, 36, 44, 45, 53, 54, 62, 63] |
| Portal Beams     | [1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 28, 29, 30, 31, 32, 37, 38, 39, 40, 41, 46, 47, 48, 49, 50, 55, 56, 57, 58, 59]     |
| Long Beams       | [66, 69, 72, 73, 76, 79, 82, 83, 216, 217, 218, 219, 220, 221, 222, 223, 99, 101, 103, 104, 106, 108, 110, 111, 113, 115, 117, 118, 120, 122, 124, 125, 224, 225, 226, 227, 228, 229, 230, 231, 141, 143, 145, 146, 148, 150, 152, 153]       |


### Secondary Members

In [ ]:
long_beams_y = group_beams_by_y(longitudinal_beams)

previous_y = None
for y,group_y in long_beams_y.items():
    base = portal.base.y if not previous_y else previous_y
    long_beams_sorted = beams_sorted_yxz(group_y)

    for beam_i in range(len(long_beams_sorted)-1):
        portal_i = beam_i%(len(portals)-1)
        if(braces_placement[portal_i]):
            long_beam_i : Beam3D = long_beams_sorted[beam_i]

            vb_1 = Beam3D(  start=Point3D(long_beam_i.start.x,
                                        base,
                                        long_beam_i.start.z),
                            end=long_beam_i.mid())
            vb_2 = Beam3D(  start=Point3D(long_beam_i.start.x,
                                        base,
                                        long_beam_i.end.z),
                            end=long_beam_i.mid())
            vertical_braces.append(vb_1)
            vertical_braces.append(vb_2)
    previous_y = y

vertical_brace_ids = add_beams(vertical_braces)

create_stub = lambda beamA,beamB : Beam3D(start=beamA.mid(),end=beamB.mid())

for long_beam_i in range(len(longitudinal_beams)-1):
    long_ii:Beam3D = longitudinal_beams[long_beam_i]
    long_ij:Beam3D = longitudinal_beams[long_beam_i+1]
    if(long_ii.start.eq_x(long_ij.start) and long_ii.end.eq_z(long_ij.end)):
        stubs.append(create_stub(long_ii,long_ij))

stubs = beams_sorted_yzx(stubs)

for stub_i in range(0, len(stubs), 2):
    if stub_i + 1 < len(stubs):  
        stub1, stub2 = stubs[stub_i], stubs[stub_i + 1]
        stub1_mid,stub2_mid = stub1.mid(),stub2.mid()
        intermediate_transverse_beams.append(Beam3D(start=stub1_mid,end=stub2_mid)\
                                                .shift_to_y(closest_to(tier_elevations,stub1_mid.mid(stub2_mid).y)))

stub_ids = add_beams(stubs)
intermediate_transverse_ids = add_beams(intermediate_transverse_beams)

portal_beams_y = group_beams_by_y([beam for portal in portals for beam in portal.beams])
for group_y in portal_beams_y.values():
    sorted_beams = beams_sorted_yzx(group_y)
    for beam_i in range(len(sorted_beams)-1):
        beam1 : Beam3D = sorted_beams[beam_i]
        beam2 : Beam3D = sorted_beams[beam_i+1]
        intermediate_long_beams.append(Beam3D(start=beam1.mid(),end=beam2.mid()))

        if(braces_placement[beam_i]):
            mid_pt = beam1.start.mid(beam2.end)
            for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                plan_braces.append(Beam3D(start=pt_x,end=mid_pt))


intermediate_long_ids = add_beams(intermediate_long_beams)
plan_brace_ids = add_beams(plan_braces)

markdown_output = ''
markdown_output = "## Secondary Members Created\n\n"
markdown_output += "| Member Types | IDs | \n"
markdown_output += "| --- | --- |\n"
markdown_output += f"| Stubs | {stub_ids} |\n"
markdown_output += f"| Intermediate Trans Beams | {intermediate_transverse_ids} |\n"
markdown_output += f"| Intermediate Long Beams | {intermediate_long_ids} |\n"
markdown_output += f"| Plan Braces | {plan_brace_ids} |\n"
markdown_output += f"| Vertical Braces | {vertical_brace_ids} |\n"

display(Markdown(markdown_output))

## Secondary Members Created

| Member Types | IDs | 
| --- | --- |
| Stubs | [234, 237, 238, 239, 242, 245, 248, 251, 252, 253, 256, 259, 261, 263, 264, 265, 267, 269, 271, 273, 274, 275, 277, 279, 281, 283, 284, 285, 287, 289, 291, 293, 294, 295, 297, 299] |
| Intermediate Trans Beams | [302, 305, 308, 311, 314, 317, 320, 323, 326, 329, 332, 335, 338, 341, 344, 347, 350, 353] |
| Intermediate Long Beams | [356, 358, 360, 362, 364, 366, 369, 371, 373, 375, 377, 379, 382, 384, 386, 388, 390, 392, 395, 397, 399, 401, 403, 405, 408, 410, 412, 414, 416, 418] |
| Plan Braces | [420, 421, 422, 423, 425, 426, 427, 428, 431, 432, 433, 434, 437, 438, 439, 440, 443, 444, 445, 446, 449, 450, 451, 452, 455, 456, 457, 458, 461, 462, 463, 464, 466, 467, 468, 469, 471, 472, 473, 474] |
| Vertical Braces | [155, 156, 158, 159, 161, 162, 164, 165, 167, 168, 170, 171, 173, 174, 176, 177, 179, 180, 182, 183, 185, 186, 188, 189, 191, 192, 194, 195, 197, 198, 200, 201] |


## Selector

### Portal Beams

In [11]:
nodes = get_node_incidences(geometry=geometry)
beam_objects:dict = get_beam_objects(geometry=geometry,property=None)
beam_nos:list = list(beam_objects.keys())